In [16]:
import torch
import torch.nn as nn
import numpy as np

class CNN(nn.Module):
    def __init__(
        self,
        input_dimension: tuple,
        number_of_filters: int,
        filter_size: tuple,
        stride: int,
        padding: int,
        max_pooling_size: tuple,
        n_neurons: int,
        n_classes: int,
        conv_activation: nn.Module,
        dense_activation: nn.Module,
        dropout_rate: float,
        use_batchnorm: bool,
        factor: float,
        dropout_organisation: int,
    ):
        super(CNN, self).__init__()

        self.conv_blocks = nn.ModuleList()
        in_channels = input_dimension[0]

        for i in range(5):
            out_channels = int((factor ** i) * number_of_filters)
            out_channels = max(out_channels, 3)

            add_dropout = (i % dropout_organisation) > 0

            conv_block = self.create_conv_block(
                in_channels,
                out_channels,
                filter_size,
                max_pooling_size,
                stride,
                padding,
                conv_activation,
                dropout_rate,
                use_batchnorm,
                add_dropout,
            )
            self.conv_blocks.append(conv_block)
            in_channels = out_channels

        self.flatten = nn.Flatten()

        # Compute the size after conv layers
        dummy_input = torch.ones(1, *input_dimension)
        with torch.no_grad():
            x = dummy_input
            for block in self.conv_blocks:
                x = block(x)
        in_features = x.view(1, -1).shape[1]

        # Define dense layers
        self.dense_block1 = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=n_neurons),
            dense_activation,
            nn.Linear(n_neurons, n_classes),
            nn.LogSoftmax(dim=1),
        )

    def create_conv_block(
        self,
        in_c,
        out_c,
        kernel_size,
        max_pooling_size,
        stride,
        padding,
        conv_activation,
        dropout_rate,
        use_batchnorm,
        add_dropout,
    ):
        layers = [
            nn.Conv2d(in_c, out_c, kernel_size=kernel_size, stride=stride, padding=padding),
            conv_activation
        ]
        if use_batchnorm:
            layers.append(nn.BatchNorm2d(out_c))
        layers.append(nn.MaxPool2d(kernel_size=max_pooling_size))
        if add_dropout:
            layers.append(nn.Dropout(p=dropout_rate))
        return nn.Sequential(*layers)

    def forward(self, x):
        for block in self.conv_blocks:
            x = block(x)
        x = self.flatten(x)
        return self.dense_block1(x)


In [17]:
import gc
import torch
import numpy as np
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder

import wandb
from torch.cuda.amp import GradScaler, autocast

In [18]:
# Device setup
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
# Dataset path
training_data_path = "/kaggle/input/nature-12k/inaturalist_12K/train"

In [20]:
torch.backends.cudnn.benchmark = True

In [31]:
# Fixed hyperparameters
config = {
    'number_of_filters': 64,
    'filter_size': 3,
    'stride': 1,
    'padding': 1,
    'max_pooling_size': 2,
    'n_neurons': 256,
    'n_classes': 10,
    'conv_activation': 'relu',
    'dense_activation': 'relu',
    'dropout_rate': 0.3,
    'use_batchnorm': True,
    'factor': 2,
    'learning_rate': 1e-3,
    'batch_size': 16,
    'epochs': 10,
    'use_augmentation': True,
    'dropout_organisation': 3
}

In [22]:
def get_transform(use_augmentation):
    if use_augmentation:
        return transforms.Compose([
            transforms.RandomCrop(50, padding=1),
            transforms.RandomGrayscale(p=0.1),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=(0, 20)),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            )
        ])
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    ])

In [30]:
sweep_config = {
    'method': 'bayes',
    'name':'PART_A_Q2_SWEEP_1',
    'metric': {'name': "val_accuracy", 'goal': 'maximize'},
    'parameters': {
        'number_of_filters': {'values': [16, 32, 64, 128, 256]},
        'filter_size': {'value': 3},
        'stride': {'value': 1},
        'padding': {'value': 1},
        'max_pooling_size': {'value': 2},
        'n_neurons': {'values': [64, 128, 256, 512, 1024]},
        'n_classes': {'value': 10},
        'conv_activation': {'values': ['relu', 'gelu', 'silu', 'mish', 'relu6', 'tanh', 'sigmoid']},
        'dense_activation': {'values': ['relu', 'gelu', 'silu', 'mish', 'relu6', 'tanh', 'sigmoid']},
        'dropout_rate': {'values': [0.2, 0.3, 0.4, 0.5]},
        'use_batchnorm': {'values': [True, False]},
        'factor': {'values': [1, 2, 3, 0.5]},
        'learning_rate': {'values': [1e-2, 1e-3, 1e-4, 1e-5]},
        'batch_size': {'value': 16},
        'epochs': {'values': [5, 10, 15]},
        'use_augmentation': {'values': [True, False]},
        'dropout_organisation': {'values': [1, 2, 3, 4, 5]},
    },
}

In [24]:
def train_model():
    # Dataset
    dataset = ImageFolder(root=training_data_path, transform=get_transform(config['use_augmentation']))
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

    # Activation options
    activations = {
        'relu': nn.ReLU(),
        'gelu': nn.GELU(),
        'silu': nn.SiLU(),
        'mish': nn.Mish(),
        'relu6': nn.ReLU6(),
        'tanh': nn.Tanh(),
        'sigmoid': nn.Sigmoid(),
    }

    # Model
    gc.collect()
    torch.cuda.empty_cache()
    model = CNN(
        input_dimension=(3, 224, 224),
        number_of_filters=config['number_of_filters'],
        filter_size=(config['filter_size'], config['filter_size']),
        stride=config['stride'],
        padding=config['padding'],
        max_pooling_size=(config['max_pooling_size'], config['max_pooling_size']),
        n_neurons=config['n_neurons'],
        n_classes=config['n_classes'],
        conv_activation=activations[config['conv_activation']],
        dense_activation=activations[config['dense_activation']],
        dropout_rate=config['dropout_rate'],
        use_batchnorm=config['use_batchnorm'],
        factor=config['factor'],
        dropout_organisation=config['dropout_organisation'],
    ).to(device)

    # Optimizer and loss
    optimizer = Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()

    # Training
    for epoch in range(config["epochs"]):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for x, y in train_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            optimizer.zero_grad()
            
            # Mixed precision training
            with autocast():
                pred = model(x)
                loss = criterion(pred, y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * x.size(0)
            correct += (pred.argmax(1) == y).sum().item()
            total += y.size(0)
            del x, y

        train_accuracy = 100 * correct / total
        avg_train_loss = train_loss / total

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
                pred = model(x)
                loss = criterion(pred, y)
                val_loss += loss.item() * x.size(0)
                correct += (pred.argmax(1) == y).sum().item()
                total += y.size(0)
                del x, y

        val_accuracy = 100 * correct / total
        avg_val_loss = val_loss / total

        print(f"Epoch [{epoch+1}/{config['epochs']}]")
        print(f"Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
        print(f"Val Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")
        print("-" * 50)

    # Save the trained model
    save_path = "/kaggle/input/nature-12k/inaturalist_12K"
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    
    return model

In [33]:
def wandb_sweep():
    with wandb.init() as run:
        sweep_config = dict(wandb.config)
        dataset = ImageFolder(root=training_data_path, transform=get_transform(sweep_config['use_augmentation']))
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_set, val_set = random_split(dataset, [train_size, val_size])
        
        # Optimized DataLoaders
        train_loader = DataLoader(
            train_set, 
            batch_size=sweep_config['batch_size'], 
            shuffle=True,
            num_workers=4,
            pin_memory=True
        )
        val_loader = DataLoader(
            val_set, 
            batch_size=sweep_config['batch_size'], 
            shuffle=False,
            num_workers=4,
            pin_memory=True
        )

        activations = {
            'relu': nn.ReLU(),
            'gelu': nn.GELU(),
            'silu': nn.SiLU(),
            'mish': nn.Mish(),
            'relu6': nn.ReLU6(),
            'tanh': nn.Tanh(),
            'sigmoid': nn.Sigmoid(),
        }
        gc.collect()
        torch.cuda.empty_cache()
        model = CNN(
            input_dimension=(3, 224, 224),
            number_of_filters=sweep_config['number_of_filters'],
            filter_size=(sweep_config['filter_size'], sweep_config['filter_size']),
            stride=sweep_config['stride'],
            padding=sweep_config['padding'],
            max_pooling_size=(sweep_config['max_pooling_size'], sweep_config['max_pooling_size']),
            n_neurons=sweep_config['n_neurons'],
            n_classes=sweep_config['n_classes'],
            conv_activation=activations[sweep_config['conv_activation']],
            dense_activation=activations[sweep_config['dense_activation']],
            dropout_rate=sweep_config['dropout_rate'],
            use_batchnorm=sweep_config['use_batchnorm'],
            factor=sweep_config['factor'],
            dropout_organisation=sweep_config['dropout_organisation'],
        ).to(device)
        optimizer = Adam(model.parameters(), lr=sweep_config['learning_rate'])
        criterion = nn.CrossEntropyLoss()
        scaler = GradScaler()
        best_val_accuracy = 0.0
        for epoch in range(sweep_config["epochs"]):
            model.train()
            train_loss = 0
            correct = 0
            total = 0
            for x, y in train_loader:
                x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
                optimizer.zero_grad()
                
                # Mixed precision training
                with autocast():
                    pred = model(x)
                    loss = criterion(pred, y)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                train_loss += loss.item() * x.size(0)
                correct += (pred.argmax(1) == y).sum().item()
                total += y.size(0)
                del x, y
            train_accuracy = 100 * correct / total
            avg_train_loss = train_loss / total
            model.eval()
            val_loss = 0
            correct = 0
            total = 0
            with torch.no_grad():
                for x, y in val_loader:
                    x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
                    pred = model(x)
                    loss = criterion(pred, y)
                    val_loss += loss.item() * x.size(0)
                    correct += (pred.argmax(1) == y).sum().item()
                    total += y.size(0)
                    del x, y
            val_accuracy = 100 * correct / total
            avg_val_loss = val_loss / total
            wandb.log({
                'epoch': epoch+1,
                'train_loss': avg_train_loss,
                'train_accuracy': train_accuracy,
                'val_loss': avg_val_loss,
                'val_accuracy': val_accuracy
            })
            print(f"Epoch [{epoch+1}/{sweep_config['epochs']}]\nTrain Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%\nVal Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%\n" + "-" * 50)
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                torch.save(model.state_dict(), "best_model_sweep.pth")
        print(f"Best model saved to best_model.pth with val_accuracy={best_val_accuracy:.2f}%")


In [34]:
if __name__ == "__main__":
    wandb.login(key='f15dba29e56f32e9c31d598bce5bc7a3c76de62e')
    sweep_id = wandb.sweep(sweep_config, project="DA6401_Assignment2")
    wandb.agent(sweep_id, function=wandb_sweep, count=20)  
    wandb.finish()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 01tppmv2
Sweep URL: https://wandb.ai/ma23c047-indian-institute-of-technology-madras/DA6401_Assignment2/sweeps/01tppmv2


wandb: Agent Starting Run: h5f54wn1 with config:
wandb: 	batch_size: 16
wandb: 	conv_activation: tanh
wandb: 	dense_activation: silu
wandb: 	dropout_organisation: 1
wandb: 	dropout_rate: 0.4
wandb: 	epochs: 5
wandb: 	factor: 0.5
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.01
wandb: 	max_pooling_size: 2
wandb: 	n_classes: 10
wandb: 	n_neurons: 1024
wandb: 	number_of_filters: 256
wandb: 	padding: 1
wandb: 	stride: 1
wandb: 	use_augmentation: True
wandb: 	use_batchnorm: True


/tmp/ipykernel_31/1913325791.py:54: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_31/1913325791.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/5]
Train Loss: 2.9309 | Train Accuracy: 10.70%
Val Loss: 2.3053 | Val Accuracy: 9.95%
--------------------------------------------------
Epoch [2/5]
Train Loss: 2.3241 | Train Accuracy: 10.00%
Val Loss: 2.3059 | Val Accuracy: 9.50%
--------------------------------------------------
Epoch [3/5]
Train Loss: 2.3102 | Train Accuracy: 10.29%
Val Loss: 2.3055 | Val Accuracy: 9.95%
--------------------------------------------------
Epoch [4/5]
Train Loss: 2.3133 | Train Accuracy: 10.43%
Val Loss: 2.3054 | Val Accuracy: 9.05%
--------------------------------------------------
Epoch [5/5]
Train Loss: 2.3086 | Train Accuracy: 10.05%
Val Loss: 2.3020 | Val Accuracy: 10.65%
--------------------------------------------------
Best model saved to best_model.pth with val_accuracy=10.65%


...

Run 23u16tuq errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_31/1913325791.py", line 51, in wandb_sweep
    ).to(device)
      ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/

Traceback (most recent call last):
  File "/tmp/ipykernel_31/1913325791.py", line 51, in wandb_sweep
    ).to(device)
      ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1326, in convert
    return t.to(
           ^^^^^
torch.OutOfMemoryError: CUDA out of memory. Tried to allocat

Run zoebx0el errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_31/1913325791.py", line 51, in wandb_sweep
    ).to(device)
      ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/

Traceback (most recent call last):
  File "/tmp/ipykernel_31/1913325791.py", line 51, in wandb_sweep
    ).to(device)
      ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1326, in convert
    return t.to(
           ^^^^^
torch.OutOfMemoryError: CUDA out of memory. Tried to allocat

Run 5aee1nm1 errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_31/1913325791.py", line 51, in wandb_sweep
    ).to(device)
      ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/